<a href="https://colab.research.google.com/github/motazsaad/infant-cry-care/blob/main/pipelineCryCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
title = 'CryCareSheet'
spreedSheet = gc.open(title)
audioContSheet = spreedSheet.sheet1

In [ ]:
import glob
import subprocess
import os
import pandas as pd
import librosa 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from gspread_dataframe import set_with_dataframe
import IPython.display as ipd
import tensorflow
import pickle


mainPath = '/content/drive/MyDrive/CryCareDataset/'
originDataPath = mainPath + 'Dataset/'
oneSecDataPath = mainPath + 'oneSecDataset/'
onSilencePath = mainPath + 'onSilence/'
donateACryPath = mainPath + 'donateacry/'
generatedData = mainPath + 'generatedData/'
exp1Path = mainPath + 'generatedData/Exp1/'
exp2Path = mainPath + 'generatedData/Exp2/'
exp3Path = mainPath + 'generatedData/Exp3/'
exp4Path = mainPath + 'generatedData/Exp4/'
exp5Path = mainPath + 'generatedData/Exp5/'
exp6Path = mainPath + 'generatedData/Exp6/'
exp7Path = mainPath + 'generatedData/Exp7/'

# directories = ['Hungry', 'lower gas', 'burp up', 'normal','pain','asphyxia', 'Uncomfortable', 'Sleepy'] 
# directories = ['Hungry', 'lower gas', 'burp up', 'Uncomfortable', 'Sleepy'] 
directories = ['Hungry', 'lower gas', 'burp up', 'normal','pain','asphyxia', 'Uncomfortable', 'Sleepy'] 

In [ ]:
def audioFeatureExtraction(raw_audio):
  X = pd.DataFrame(columns = np.arange(45), dtype = 'float32').astype(np.float32)
  for i, filename in enumerate(raw_audio):
      if filename.endswith(".wav"):
          audiofile, sr = librosa.load(filename, sr = 11025)
          fingerprint= librosa.feature.spectral_centroid(audiofile, 
                                            sr=sr)
          x = pd.DataFrame(fingerprint, dtype = 'float32')
          x[44] = raw_audio[filename]
          X.loc[i] = x.loc[0]
  X = X.fillna(0)
  y = X[44]
  del X[44]
  X = X.astype(np.float32)
  return X, y

In [ ]:
def exportClassificationReport(report, sheetName, directories):
  df = pd.DataFrame(report).transpose()
  df[''] = directories + ['accuracy', 'macro avg', 'weighted avg']
  try:
    spreedSheet.add_worksheet(title =sheetName, rows = 200, cols = 100)
  except:
    print('file exist')
  reportSheet = spreedSheet.worksheet(sheetName)
  set_with_dataframe(reportSheet, df) 
  # return df

In [ ]:
def createRawAudio(path, directories) :
  print(path)
  raw_audio = dict()
  for directory in directories :
    for filename in glob.glob(path + directory + '/**.wav'):
      raw_audio[filename] = directory
  return raw_audio

In [ ]:
def makeModel(path, classifier, rawAudioFunction, directories):
  raw_audio_test = rawAudioFunction(path + 'output/val/', directories)
  X_test, y_test = audioFeatureExtraction(raw_audio_test)

  return  X_test, y_test 
  #  model = get_scores(classifier, X_train, X_test, y_train, y_test)
  # return (model)

X_test, y_test  = makeModel(exp4Path,  RandomForestClassifier, createRawAudio, directories)

/content/drive/MyDrive/CryCareDataset/generatedData/Exp4/output/val/


In [ ]:
asphyxia_model_file = mainPath + 'asphyxia_model.sav'
hungry_model_file = mainPath + 'hungry_model.sav'
final_model_file = mainPath + 'final_model.sav'

asphyxia_model = pickle.load(open(asphyxia_model_file, 'rb'))
hungry_model = pickle.load(open(hungry_model_file, 'rb'))
final_model = pickle.load(open(final_model_file, 'rb'))

In [ ]:
y_predict=[]
for i in range(len(X_test)):

    result =  asphyxia_model.predict(X_test.iloc[[i]])
    if result == 'Not asphyxia':
      result2 = hungry_model.predict(X_test.iloc[[i]])
      if result2 =='Not Hungry':
        result3 = final_model.predict(X_test.iloc[[i]])
        y_predict.append(result3[0])
      else:
        y_predict.append(result2[0])
    else:
      y_predict.append(result[0])

In [ ]:
report = classification_report(y_test, pd.DataFrame(y_predict), target_names=directories, output_dict=True)
exportClassificationReport(report, 'pipeline_model2', directories)
print(report)

{'Hungry': {'precision': 0.797153024911032, 'recall': 0.6086956521739131, 'f1-score': 0.6902927580893683, 'support': 368}, 'lower gas': {'precision': 0.625531914893617, 'recall': 0.786096256684492, 'f1-score': 0.6966824644549764, 'support': 187}, 'burp up': {'precision': 0.6038961038961039, 'recall': 0.7153846153846154, 'f1-score': 0.6549295774647886, 'support': 130}, 'normal': {'precision': 0.8518518518518519, 'recall': 0.6494117647058824, 'f1-score': 0.7369826435246997, 'support': 425}, 'pain': {'precision': 0.6506024096385542, 'recall': 0.5510204081632653, 'f1-score': 0.5966850828729281, 'support': 98}, 'asphyxia': {'precision': 0.7076923076923077, 'recall': 0.6133333333333333, 'f1-score': 0.657142857142857, 'support': 75}, 'Uncomfortable': {'precision': 0.5468277945619335, 'recall': 0.7125984251968503, 'f1-score': 0.6188034188034187, 'support': 254}, 'Sleepy': {'precision': 0.618421052631579, 'recall': 0.7833333333333333, 'f1-score': 0.6911764705882353, 'support': 240}, 'accuracy':